In [1]:
import pandas as pd
import numpy as np
import sys
import os
import bs4
from IPython.display import display_html,clear_output, HTML
import re
from datetime import datetime
import ast
import itertools
from tqdm import tqdm,trange
from file_tools import *
from request_tools import *
from parse_tools import *
from load_tools import *
from model_tools import *

pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 30)
pd.set_option('compute.use_numexpr', False)

import warnings
from pandas.errors import PerformanceWarning
from sklearn.exceptions import DataConversionWarning, ConvergenceWarning
pd.options.mode.use_inf_as_na = True
warnings.filterwarnings('ignore', category=PerformanceWarning)
warnings.filterwarnings('ignore', category=DataConversionWarning)
warnings.filterwarnings('ignore', category=ConvergenceWarning)

import pandas as pd
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif, RFE, mutual_info_regression
from sklearn.linear_model import LogisticRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from scipy.stats import pearsonr
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVR

In [6]:
# Load the predictions
PREDS_BY_PLYR = pd.read_csv('./08-experiments/03-boxscores-players-stats-1st-vs-2nd-half-1997-2023/predict-players-pts-total/by-players/20240118-224847-experiment*/predictions.csv',
                            index_col=[0,1,2,3],header=[0,1]).iloc[:5000,:]
PREDS_BY_PLYR

y_true   
                                                                                                      Unnamed: 4_level_1   
                          boxscore_id                  team_id              player_id                                      
/players/a/abdulma02.html /boxscores/199611010HOU.html /teams/SAC/1997.html /players/a/abdulma02.html               17.0  \
                          /boxscores/199611020DAL.html /teams/SAC/1997.html /players/a/abdulma02.html                2.0   
                          /boxscores/199611050SAC.html /teams/SAC/1997.html /players/a/abdulma02.html               16.0   
                          /boxscores/199611070SAC.html /teams/SAC/1997.html /players/a/abdulma02.html                0.0   
                          /boxscores/199611110SAC.html /teams/SAC/1997.html /players/a/abdulma02.html                7.0   
                          /boxscores/199611130UTA.html /teams/SAC/1997.html /players/a/abdulma02.html               11.0   
                          /boxscores/199611150SAC.html /teams/SAC/1997.html /players/a/abdulma02.html               22.0   
                          /boxscores/199611160SEA.html /teams/SAC/1997.html /players/a/abdulma02.html               10.0   
                          /boxscores/199611180VAN.html /teams/SAC/1997.html /players/a/abdulma02.html               10.0   
                          /boxscores/199611190POR.html /teams/SAC/1997.html /players/a/abdulma02.html               31.0   
                          /boxscores/199611220SAC.html /teams/SAC/1997.html /players/a/abdulma02.html               23.0   
                          /boxscores/199611240CLE.html /teams/SAC/1997.html /players/a/abdulma02.html                5.0   
                          /boxscores/199611260TOR.html /teams/SAC/1997.html /players/a/abdulma02.html               18.0   
                          /boxscores/199611280IND.html /teams/SAC/1997.html /players/a/abdulma02.html               12.0   
                          /boxscores/199611300NYK.html /teams/SAC/1997.html /players/a/abdulma02.html               17.0   
...                                                                                                                  ...   
/players/a/allenra02.html /boxscores/199611150SAC.html /teams/MIL/1997.html /players/a/allenra02.html                9.0   
                          /boxscores/199611170LAC.html /teams/MIL/1997.html /players/a/allenra02.html               23.0   
                          /boxscores/199611210MIL.html /teams/MIL/1997.html /players/a/allenra02.html               13.0   
                          /boxscores/199611250ORL.html /teams/MIL/1997.html /players/a/allenra02.html               12.0   
                          /boxscores/199611300MIL.html /teams/MIL/1997.html /players/a/allenra02.html               15.0   
                          /boxscores/199612030MIL.html /teams/MIL/1997.html /players/a/allenra02.html                8.0   
                          /boxscores/199612070WSB.html /teams/MIL/1997.html /players/a/allenra02.html               12.0   
                          /boxscores/199612080MIL.html /teams/MIL/1997.html /players/a/allenra02.html               17.0   
                          /boxscores/199612100MIL.html /teams/MIL/1997.html /players/a/allenra02.html               16.0   
                          /boxscores/199612120MIL.html /teams/MIL/1997.html /players/a/allenra02.html               17.0   
                          /boxscores/199612140MIL.html /teams/MIL/1997.html /players/a/allenra02.html               11.0   
                          /boxscores/199612160BOS.html /teams/MIL/1997.html /players/a/allenra02.html               16.0   
                          /boxscores/199612180MIL.html /teams/MIL/1997.html /players/a/allenra02.html                7.0   
                          /boxscores/199612190TOR.html /teams/MIL/1997.html /players/a/allenra02.html               12.0   
                      

In [7]:
PREDS_VAL = get_predictions_validation_set(PREDS_BY_PLYR)
PREDS_VAL

y_true   
                          boxscore_id                  team_id              player_id                           
/players/a/abdulma02.html /boxscores/199611010HOU.html /teams/SAC/1997.html /players/a/abdulma02.html    17.0  \
                          /boxscores/199611020DAL.html /teams/SAC/1997.html /players/a/abdulma02.html     2.0   
                          /boxscores/199611050SAC.html /teams/SAC/1997.html /players/a/abdulma02.html    16.0   
                          /boxscores/199611070SAC.html /teams/SAC/1997.html /players/a/abdulma02.html     0.0   
                          /boxscores/199611110SAC.html /teams/SAC/1997.html /players/a/abdulma02.html     7.0   
                          /boxscores/199611130UTA.html /teams/SAC/1997.html /players/a/abdulma02.html    11.0   
                          /boxscores/199611150SAC.html /teams/SAC/1997.html /players/a/abdulma02.html    22.0   
                          /boxscores/199611160SEA.html /teams/SAC/1997.html /players/a/abdulma02.html    10.0   
                          /boxscores/199611180VAN.html /teams/SAC/1997.html /players/a/abdulma02.html    10.0   
                          /boxscores/199611190POR.html /teams/SAC/1997.html /players/a/abdulma02.html    31.0   
                          /boxscores/199611220SAC.html /teams/SAC/1997.html /players/a/abdulma02.html    23.0   
                          /boxscores/199611240CLE.html /teams/SAC/1997.html /players/a/abdulma02.html     5.0   
                          /boxscores/199611260TOR.html /teams/SAC/1997.html /players/a/abdulma02.html    18.0   
                          /boxscores/199611280IND.html /teams/SAC/1997.html /players/a/abdulma02.html    12.0   
                          /boxscores/199611300NYK.html /teams/SAC/1997.html /players/a/abdulma02.html    17.0   
...                                                                                                       ...   
/players/a/allenra02.html /boxscores/199611150SAC.html /teams/MIL/1997.html /players/a/allenra02.html     9.0   
                          /boxscores/199611170LAC.html /teams/MIL/1997.html /players/a/allenra02.html    23.0   
                          /boxscores/199611210MIL.html /teams/MIL/1997.html /players/a/allenra02.html    13.0   
                          /boxscores/199611250ORL.html /teams/MIL/1997.html /players/a/allenra02.html    12.0   
                          /boxscores/199611300MIL.html /teams/MIL/1997.html /players/a/allenra02.html    15.0   
                          /boxscores/199612030MIL.html /teams/MIL/1997.html /players/a/allenra02.html     8.0   
                          /boxscores/199612070WSB.html /teams/MIL/1997.html /players/a/allenra02.html    12.0   
                          /boxscores/199612080MIL.html /teams/MIL/1997.html /players/a/allenra02.html    17.0   
                          /boxscores/199612100MIL.html /teams/MIL/1997.html /players/a/allenra02.html    16.0   
                          /boxscores/199612120MIL.html /teams/MIL/1997.html /players/a/allenra02.html    17.0   
                          /boxscores/199612140MIL.html /teams/MIL/1997.html /players/a/allenra02.html    11.0   
                          /boxscores/199612160BOS.html /teams/MIL/1997.html /players/a/allenra02.html    16.0   
                          /boxscores/199612180MIL.html /teams/MIL/1997.html /players/a/allenra02.html     7.0   
                          /boxscores/199612190TOR.html /teams/MIL/1997.html /players/a/allenra02.html    12.0   
                          /boxscores/199612210MIL.html /teams/MIL/1997.html /players/a/allenra02.html    16.0   

                                                                                                       AdaBoostRegressor   
                          boxscore_id                  team_id              player_id                                      
/players/a/abdulma02.html /boxscores/199611010HOU.html /teams/SAC/1997.html /players/a/abdulma02.html          19.444444  \
 

![](./__images__/18_001.jpg)

In [64]:
# scoring_func = lambda y_true,y_pred: (y_pred/15) * np.exp(-1/2 * (((y_true-y_pred)-0.5)/(y_pred/15))**2)
scoring_func = lambda y_true,y_pred: ((y_pred+0.5)/18) * np.exp(-((np.log(y_true-y_pred+1)-0.4)**2/((y_pred+0.5)/18)**2))
for col in PREDS_VAL.columns[1:]:
    print(col,scoring_func(PREDS_VAL['y_true'],PREDS_VAL[col]).mean())

AdaBoostRegressor 0.33542784435734396
LinearRegression -4516232.364230117
MLPRegressor 0.23948612880416412
RandomForestRegressor 0.320975678823621
Ridge 0.3085874652788106
SVR 0.28685584202629505


/Users/jasetran/Jase/UM/Git/basketball-analytics/.venv/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/jasetran/Jase/UM/Git/basketball-analytics/.venv/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [63]:
pd.concat([PREDS_VAL['y_true'],PREDS_VAL[col],(PREDS_VAL[col]).astype(int).rename('round') - 0.5,scoring_func(PREDS_VAL['y_true'],(PREDS_VAL[col]).astype(int)-0.5).round(4)],axis=1).sort_values([0,'round'],ascending=False).head(50)

/Users/jasetran/Jase/UM/Git/basketball-analytics/.venv/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


y_true   
                          boxscore_id                  team_id              player_id                           
/players/a/aldrila01.html /boxscores/201903020SAS.html /teams/SAS/2019.html /players/a/aldrila01.html    27.0  \
                          /boxscores/201101140PHO.html /teams/POR/2011.html /players/a/aldrila01.html    25.0   
                          /boxscores/201902020SAS.html /teams/SAS/2019.html /players/a/aldrila01.html    25.0   
/players/a/abdursh01.html /boxscores/199712030VAN.html /teams/VAN/1998.html /players/a/abdursh01.html    24.0   
/players/a/aldrila01.html /boxscores/200902200POR.html /teams/POR/2009.html /players/a/aldrila01.html    24.0   
                          /boxscores/200911230POR.html /teams/POR/2010.html /players/a/aldrila01.html    24.0   
                          /boxscores/201904050WAS.html /teams/SAS/2019.html /players/a/aldrila01.html    24.0   
/players/a/abdursh01.html /boxscores/200004110GSW.html /teams/VAN/2000.html /players/a/abdursh01.html    23.0   
/players/a/aldrila01.html /boxscores/200804030POR.html /teams/POR/2008.html /players/a/aldrila01.html    23.0   
                          /boxscores/200810310POR.html /teams/POR/2009.html /players/a/aldrila01.html    23.0   
                          /boxscores/201201270POR.html /teams/POR/2012.html /players/a/aldrila01.html    23.0   
                          /boxscores/201002030UTA.html /teams/POR/2010.html /players/a/aldrila01.html    27.0   
                          /boxscores/202201190WAS.html /teams/BRK/2022.html /players/a/aldrila01.html    27.0   
/players/a/abdursh01.html /boxscores/200102230MIL.html /teams/VAN/2001.html /players/a/abdursh01.html    22.0   
                          /boxscores/200201170ATL.html /teams/ATL/2002.html /players/a/abdursh01.html    22.0   
                          /boxscores/200303020MIL.html /teams/ATL/2003.html /players/a/abdursh01.html    22.0   
                          /boxscores/200303080ATL.html /teams/ATL/2003.html /players/a/abdursh01.html    22.0   
                          /boxscores/200411050POR.html /teams/POR/2005.html /players/a/abdursh01.html    22.0   
                          /boxscores/200503090POR.html /teams/POR/2005.html /players/a/abdursh01.html    22.0   
/players/a/aldrila01.html /boxscores/200901190POR.html /teams/POR/2009.html /players/a/aldrila01.html    22.0   
                          /boxscores/201001180WAS.html /teams/POR/2010.html /players/a/aldrila01.html    22.0   
                          /boxscores/201002100PHO.html /teams/POR/2010.html /players/a/aldrila01.html    22.0   
                          /boxscores/201002160POR.html /teams/POR/2010.html /players/a/aldrila01.html    22.0   
                          /boxscores/201003140POR.html /teams/POR/2010.html /players/a/aldrila01.html    22.0   
                          /boxscores/201203070MIN.html /teams/POR/2012.html /players/a/aldrila01.html    22.0   
                          /boxscores/201311090SAC.html /teams/POR/2014.html /players/a/aldrila01.html    22.0   
                          /boxscores/201612180SAS.html /teams/SAS/2017.html /players/a/aldrila01.html    22.0   
                          /boxscores/201712200POR.html /teams/SAS/2018.html /players/a/aldrila01.html    22.0   
                          /boxscores/201903040SAS.html /teams/SAS/2019.html /players/a/aldrila01.html    22.0   
                          /boxscores/202101180POR.html /teams/SAS/2021.html /players/a/aldrila01.html    22.0   
/players/a/abdursh01.html /boxscores/200311220ATL.html /teams/ATL/2004.html /players/a/abdursh01.html    26.0   
                          /boxscores/199612280VAN.html /teams/VAN/1997.html /players/a/abdursh01.html    21.0   
                          /boxscores/199703270VAN.html /teams/VAN/1997.html /players/a/abdursh01.html    21.0   
                          /boxscores/199712090PHO.html /teams/VAN/1998.html /players/a/abdursh01.html    21.0   
                          /boxscore

In [18]:
# Round predictions to nearest 0.5
(PREDS_VAL['SVR']).astype(int) - 0.5

                           boxscore_id                   team_id               player_id                
/players/a/abdulma02.html  /boxscores/199611010HOU.html  /teams/SAC/1997.html  /players/a/abdulma02.html    15.5
                           /boxscores/199611020DAL.html  /teams/SAC/1997.html  /players/a/abdulma02.html    14.5
                           /boxscores/199611050SAC.html  /teams/SAC/1997.html  /players/a/abdulma02.html    14.5
                           /boxscores/199611070SAC.html  /teams/SAC/1997.html  /players/a/abdulma02.html    14.5
                           /boxscores/199611110SAC.html  /teams/SAC/1997.html  /players/a/abdulma02.html    14.5
                           /boxscores/199611130UTA.html  /teams/SAC/1997.html  /players/a/abdulma02.html    15.5
                           /boxscores/199611150SAC.html  /teams/SAC/1997.html  /players/a/abdulma02.html    14.5
                           /boxscores/199611160SEA.html  /teams/SAC/1997.html  /players/a/abdulma02.html

In [31]:
PREDS_VAL['y_true'].describe()

count    5000.000000
mean       14.745600
std         7.593314
min         0.000000
25%         9.000000
50%        14.000000
75%        20.000000
max        56.000000
Name: y_true, dtype: float64